## Cleaning and Validation

This is the first in a series of notebooks that make up a [case study in exploratory data analysis](https://allendowney.github.io/PoliticalAlignmentCaseStudy/). This case study is part of the [Elements of Data Science](https://allendowney.github.io/ElementsOfDataScience/) curriculum.


In this notebook, we:

1. Read data from the General Social Survey (GSS),
2. Clean the data, particularly dealing with special codes that indicate missing data,
3. Validate the data by comparing the values in the dataset with values documented in the codebook.
4. Generate resampled datasets that correct for deliberate oversampling in the dataset, and
5. Store the resampled data in a binary format (HDF5) that makes it easier to work with in the notebooks that follow this one.

In [1]:
import pandas as pd
import numpy as np

### Reading the data

The data we'll use is from the General Social Survey (GSS). Using the [GSS Data Explorer](https://gssdataexplorer.norc.org), A subset of the variables in the GSS were selected and made  available along with this notebook. The following cell loads this extract.

In [2]:
gss = pd.read_hdf("../data/gss_pacs_2022.hdf", "gss")
gss.shape

(72390, 207)

We can use `head` to see what the `DataFrame` looks like.


In [3]:
gss.head()

,abany,abdefect,abhlth,abnomore,abpoor,abrape,absingle,acqntsex,adults,affrmact,...,trdunion,trust,union,wkharsex,wkracism,wksexism,wtssall,wtssps,xmarsex,year
0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,0.4446,0.663196,NaN,1972
1,NaN,1.0,1.0,2.0,2.0,1.0,1.0,NaN,2.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,0.8893,0.917370,NaN,1972
2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2.0,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,0.8893,0.897413,NaN,1972
3,NaN,2.0,1.0,2.0,1.0,1.0,1.0,NaN,2.0,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,0.8893,1.066341,NaN,1972
4,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2.0,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,0.8893,0.944324,NaN,1972


This dataset has 72390 rows, one for each respondent, and 207 columns, one for each variable.


### Validation

Now that we've got the data loaded, it is important to validate it, which means checking for errors. The kinds of errors you have to check for depend on the nature of the data, the collection process, how the data is stored and transmitted, etc. For this dataset, there are three kinds of validation we'll think about:

1. We need to check the **integrity** of the dataset; that is, whether the data were corrupted or changed during transmission, storage, or conversion from one format to another.
2. We need to check our **interpretation** of the data; for example, whether the numbers used to encode the data mean what we think they mean.
3. We will also keep an eye out for **invalid** data; for example, missing data might be represented using special codes, or there might be patterns in the data that indicate problems with the survey process and the recording of the data.

In a different dataset I worked with, I found a surprising number of respondents whose height was supposedly 62 centimeters. After investigating, I concluded that they were probably 6 feet, 2 inches, and their heights were recorded incorrectly. Validating data can be a tedious process, but it is important. If you interpret data incorrectly and publish invalid results, you will be embarrassed in the best case, and in the worst case you might do real harm.

The first variable we'll validate is called polviews. It records responses to the following question:
> We hear a lot of talk these days about liberals and conservatives. I'm going to show you a seven-point scale on which the political views that people might hold are arranged from extremely liberal--point 1--to extremely conservative--point 7. Where would you place yourself on this scale?

You can read the documentation of this variable in the [GSS codebook](https://gssdataexplorer.norc.org/variables/178/vshow). The responses are encoded like this:

|   |                        |
|---|------------------------|
| 1 | Extremely liberal      |
| 2 | Liberal                |
| 3 | Slightly liberal       |
| 4 | Moderate               |
| 5 | Slghtly conservative   |
| 6 | Conservative           |
| 7 | Extremely conservative |
| 8 | Don't know             |
| 9 | No answer              |
| 0 | Not applicable         |


The following function, `values`, takes a Series that represents a single variable and returns the values in the series and their frequencies.

In [4]:
def values(series):
    """Count the values and sort.

    series: pd.Series

    returns: series mapping from values to frequencies
    """
    return series.value_counts().sort_index()

Here are the values for the variable `polviews`.

In [5]:
polviews = gss["polviews"]
values(polviews)

polviews
1.0     2081
2.0     7623
3.0     7900
4.0    23992
5.0     9596
6.0     9361
7.0     2165
Name: count, dtype: int64

To check the integrity of the data and confirm that we have loaded it correctly, we'll do a "spot check"; that is, we'll pick one year and compare the values we see in the dataset to the values reported in the codebook.

We can select values from a single year like this:

In [6]:
one_year = gss["year"] == 1974
values(polviews[one_year])

polviews
1.0     22
2.0    201
3.0    207
4.0    564
5.0    221
6.0    160
7.0     35
Name: count, dtype: int64

If you compare these results to the [values in the codebook](https://gssdataexplorer.norc.org/variables/178/vshow), you should see that they agree.

### Missing data

For many variables, missing values are encoded with numerical codes that we need to replace before we do any analysis. For `polviews`, the values 8, 9, and 0 represent "Don't know", "No answer", and "Not applicable". "Not applicable" usually means the respondent was not asked a particular question.

To keep things simple, we'll treat all of these values as equivalent, but we lose some information by doing that. For example, if a respondent refuses to answer a question, that might suggest something about their answer. If so, treating their response as missing data might bias the results. Fortunately, for most questions the number of respondents who refused to answer is small. I'll replace the numeric codes 8, 9, and 0 with `np.nan`, which is a special value used to indicate missing data.

In [7]:
NA = np.nan
clean = polviews.replace([0, 8, 9], NA)
clean

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
72385    1.0
72386    4.0
72387    3.0
72388    3.0
72389    2.0
Name: polviews, Length: 72390, dtype: float64

When `np.nan` is displayed, it appears as `NaN`, which stands for "not a number". We can use `notna` and `sum` to count the valid responses:

In [8]:
valid_responses_count = clean.notna().sum()
print(valid_responses_count)

62718


And we use isna to count the missing responses:


In [9]:
missing_responses_count = clean.isna().sum()
print(missing_responses_count)

9672


### Replacing missing data

For the other variables in this dataset, I read through the code book and identified the special values that indicate missing data. I recorded that information in the following function, which is intended to replace special values with `NaN`.

In [10]:
def replace_invalid(df, columns, bad):
    for column in columns:
        df.replace({column: NA}, inplace=True)

In [11]:
def gss_replace_invalid(df):
    """Replace invalid data with NaN.

    df: DataFrame
    """
    # different variables use different codes for invalid data
    df.replace({"cohort": NA}, inplace=True)

    # since there are a lot of variables that use 0, 8, and 9 for invalid data,
    # I'll use a loop to replace all of them
    columns = [
        "abany",
        "abdefect",
        "abhlth",
        "abnomore",
        "abpoor",
        "abrape",
        "absingle",
        "acqntsex",
        "affrmact",
        "bible",
        "cappun",
        "colath",
        "colcom",
        "colhomo",
        "colmil",
        "colmslm",
        "colrac",
        "colsoc",
        "compuse",
        "conarmy",
        "conbus",
        "conclerg",
        "coneduc",
        "confed",
        "confinan",
        "conjudge",
        "conlabor",
        "conlegis",
        "conmedic",
        "conpress",
        "consci",
        "contv",
        "databank",
        "discaffm",
        "discaffw",
        "divlaw",
        "divorce",
        "eqwlth",
        "fair",
        "fear",
        "fechld",
        "fefam",
        "fehelp",
        "fehire",
        "fehome",
        "fejobaff",
        "fepol",
        "fepres",
        "fepresch",
        "fework",
        "finrela",
        "frndsex",
        "fund",
        "god",
        "goodlife",
        "grass",
        "gunlaw",
        "hapmar",
        "happy",
        "health",
        "helpful",
        "hhrace",
        "homosex",
        "hunt",
        "libath",
        "libcom",
        "libhomo",
        "libmil",
        "libmslm",
        "librac",
        "libsoc",
        "life",
        "matesex",
        "meovrwrk",
        "miracles",
        "nataid",
        "natarms",
        "natchld",
        "natcity",
        "natcrime",
        "natdrug",
        "nateduc",
        "natenrgy",
        "natenvir",
        "natfare",
        "natheal",
        "natmass",
        "natpark",
        "natrace",
        "natroad",
        "natsci",
        "natsoc",
        "natspac",
        "othersex",
        "paidsex",
        "pikupsex",
        "polabuse",
        "polattak",
        "polescap",
        "polhitok",
        "polmurdr",
        "polviews",
        "popespks",
        "pornlaw",
        "postlife",
        "pray",
        "prayer",
        "premarsx",
        "pres00",
        "pres04",
        "pres08",
        "pres12",
        "pres96",
        "racchurh",
        "racclos",
        "racdif1",
        "racdif2",
        "racdif3",
        "racdif4",
        "racdin",
        "racdis",
        "racfew",
        "rachaf",
        "rachome",
        "racinteg",
        "raclive",
        "racmar",
        "racmost",
        "racopen",
        "racpres",
        "racpush",
        "racschol",
        "racseg",
        "racwork",
        "reborn",
        "relexp",
        "relexper",
        "reliten",
        "relpersn",
        "res16",
        "rowngun",
        "satfin",
        "satjob",
        "savesoul",
        "sexbirth",
        "sexeduc",
        "sexnow",
        "sexornt",
        "sexsex",
        "sexsex5",
        "spanking",
        "spkath",
        "spkcom",
        "spkhomo",
        "spklang",
        "spkmil",
        "spkmslm",
        "spkrac",
        "spksoc",
        "sprtprsn",
        "teensex",
        "trdunion",
        "trust",
        "union",
        "wkharsex",
        "wkracism",
        "wksexism",
        "xmarsex",
        "commun",
    ]
    replace_invalid(df, columns, [0, 8, 9])

    columns = ["degree", "partyid"]
    replace_invalid(df, columns, [8, 9])

    df["phone"] = df["phone"].replace([0, 2, 9], NA)
    df["owngun"] = df["owngun"].replace([0, 3, 8, 9], NA)
    df["pistol"] = df["pistol"].replace([0, 3, 8, 9], NA)
    df["class"] = df["class"].replace([0, 5, 8, 9], NA)

    df["chldidel"] = df["chldidel"].replace([-1, 8, 9], NA)
    df["sexfreq"] = df["sexfreq"].replace([-1, 8, 9], NA)

    df["attend"] = df["attend"].replace([9], NA)
    df["childs"] = df["childs"].replace([9], NA)
    df["adults"] = df["adults"].replace([9], NA)

    df["age"] = df["age"].replace([0, 98, 99], NA)
    df["relig"] = df["relig"].replace([0, 98, 99], NA)
    df["relig16"] = df["relig16"].replace([0, 98, 99], NA)
    df["relactiv"] = df["relactiv"].replace([0, 98, 99], NA)

    # note: sibs contains some unlikely numbers
    df["sibs"] = df["sibs"].replace([-1, 98, 99], NA)
    df["hrsrelax"] = df["hrsrelax"].replace([-1, 98, 99], NA)

    df["educ"] = df["educ"].replace([97, 98, 99], NA)

    df["realinc"] = df["realinc"].replace([0], NA)
    df["realrinc"] = df["realrinc"].replace([0], NA)

    df["income"] = df["income"].replace([0, 13, 98, 99], NA)
    df["rincome"] = df["rincome"].replace([0, 13, 98, 99], NA)

In [12]:
gss_replace_invalid(gss)

In [13]:
# check that we've cleaned all columns that need it;
# all columns we've added NaN to should be floats

# some columns have no missing values
clean_columns = ["id", "year", "ballot", "sex", "race", "reg16", "region", "srcbelt"]

for column in gss.columns:
    if gss[column].dtype == int and column not in clean_columns:
        print(f"'{column}', ", end="")
print()

### Resampling

The GSS uses stratified sampling, which means that some groups are deliberately oversampled to help with statistical validity. As a result, each respondent has a sampling weight which is proportional to the number of people in the population they represent.

Before running any analysis, we can compensate for stratified sampling by "resampling", that is, by drawing a random sample from the dataset, where each respondent's chance of appearing in the sample is proportional to their sampling weight.

In [14]:
def resample_rows_weighted(df, column):
    """Resamples a DataFrame using probabilities proportional to given column.

    df: DataFrame
    column: string column name to use as weights

    returns: DataFrame
    """
    weights = df[column]
    sample = df.sample(n=len(df), replace=True, weights=weights)
    return sample

In [15]:
def resample_by_year(df, column):
    """Resample rows within each year.

    df: DataFrame
    column: string name of weight variable

    returns DataFrame
    """
    grouped = df.groupby("year")
    samples = [resample_rows_weighted(group, column) for _, group in grouped]
    sample = pd.concat(samples, ignore_index=True)
    return sample

In [16]:
np.random.seed(19)
sample = resample_by_year(gss, "wtssall")

### Saving the results

I'll save the results to an HDF5 file, which is a binary format that makes it much faster to read the data back. An HDF5 file is like a dictionary on disk. It contains keys and corresponding values. The `to_hdf` method takes three arguments: The filename, `gss_pacs_clean.hdf`; The key, `gss`; And the compression level, which controls how hard the algorithm works to compress the file.

So this file contains a single key, `gss`, which maps to the DataFrame with the original GSS data. The argument `w` says that if the file already exists, we should overwrite it. With compression level `6`, it reduces the size of the file by a factor of more than 10.

In [17]:
# if the file already exists, remove it
from pathlib import Path

if Path.is_file(Path("../data/gss_pacs_2022_clean.hdf")):
    ! rm -v ../data/gss_pacs_2022_resampled.hdf

../data/gss_pacs_2022_resampled.hdf


In [18]:
# save the original

gss.to_hdf("../data/gss_pacs_2022_clean.hdf", key="gss", mode="w", complevel=6)

In [19]:
! ls -lh ../data/gss_pacs_2022_clean.hdf

-rw-r--r--  1 wilderbarbosagalvao  staff    10M Aug 22 18:32 ../data/gss_pacs_2022_clean.hdf


And I'll create a second file with three random resamplings of the original dataset.

In [20]:
# generate and store three resamplings
keys = ["gss0", "gss1", "gss2"]

for i in range(3):
    np.random.seed(i)
    sample = resample_by_year(gss, "wtssall")
    sample.to_hdf("../data/gss_pacs_2022_resampled.hdf", key=keys[i], complevel=6)

In [21]:
! ls -lh ../data/gss_pacs_2022_resampled.hdf

-rw-r--r--  1 wilderbarbosagalvao  staff    31M Aug 22 18:32 ../data/gss_pacs_2022_resampled.hdf


For the other notebooks in this case study, we'll load this resampled data rather than reading and cleaning the data every time.